## Nomic 多模态 RAG

许多文档包含混合内容类型,包括文本和图像。

然而,在大多数 RAG 应用程序中,图像中包含的信息都被丢失了。

随着多模态 LLM 的出现,例如 [GPT-4V](https://openai.com/research/gpt-4v-system-card),值得考虑如何在 RAG 中利用图像:

在这个演示中我们:

* 使用来自 Nomic Embed 的多模态嵌入模型[Vision](https://huggingface.co/nomic-ai/nomic-embed-vision-v1.5)和[Text](https://huggingface.co/nomic-ai/nomic-embed-text-v1.5)来嵌入图像和文本
* 使用相似度搜索来检索两者
* 将原始图像和文本块传递给多模态 LLM 进行答案合成

## 注册

获取 API 令牌,然后运行:
```
! nomic login
```

然后使用生成的 API 令牌运行
```
! nomic login < token >
```

## 包

对于 `unstructured`,您还需要在系统中安装 `poppler`([安装说明](https://pdf2image.readthedocs.io/en/latest/installation.html))和 `tesseract`([安装说明](https://tesseract-ocr.github.io/tessdoc/Installation.html))。

In [ ]:
! nomic login token

In [ ]:
! pip install -U langchain-nomic langchain-chroma langchain-community tiktoken langchain-openai langchain # (newest versions required for multi-modal)

In [ ]:
# lock to 0.10.19 due to a persistent bug in more recent versions
! pip install "unstructured[all-docs]==0.10.19" pillow pydantic lxml pillow matplotlib tiktoken

## 数据加载

### 分割 PDF 文本和图像
  
让我们看一个包含有趣图像的 PDF 示例。

1/ 来自 J Paul Getty 博物馆的艺术品:

 * 这里有一个 [zip 文件](https://drive.google.com/file/d/18kRKbq2dqAhhJ3DfZRnYcTBEUfYxe1YR/view?usp=sharing),其中包含 PDF 和已提取的图像。
* https://www.getty.edu/publications/resources/virtuallibrary/0892360224.pdf

2/ 来自国会图书馆的著名照片:

* https://www.loc.gov/lcm/pdf/LCM_2020_1112.pdf
* 我们将在下面使用这个作为示例

我们可以使用 [Unstructured](https://unstructured-io.github.io/unstructured/introduction.html#key-concepts) 中的 `partition_pdf` 来提取文本和图像。

要提取图像,请提供以下参数:
```
extract_images_in_pdf=True
```



如果使用此 zip 文件,则可以仅处理文本:
```
extract_images_in_pdf=False
```

In [ ]:
# 包含 PDF 和提取图像的文件夹
from pathlib import Path

# 替换为实际图像路径
path = Path("../art")

In [ ]:
path.resolve()

In [ ]:
# 提取图像、表格和分块文本
from unstructured.partition.pdf import partition_pdf

raw_pdf_elements = partition_pdf(
    filename=str(path.resolve()) + "/getty.pdf",
    extract_images_in_pdf=False,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)

In [ ]:
# 按类型分类文本元素
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

## 使用我们的文档进行多模态嵌入

我们将使用 [nomic-embed-vision-v1.5](https://huggingface.co/nomic-ai/nomic-embed-vision-v1.5) 嵌入模型。此模型与 [nomic-embed-text-v1.5](https://huggingface.co/nomic-ai/nomic-embed-text-v1.5) 对齐,允许进行多模态语义搜索和多模态 RAG!

In [ ]:
import os
import uuid

import chromadb
import numpy as np
from langchain_chroma import Chroma
from langchain_nomic import NomicEmbeddings
from PIL import Image as _PILImage

# 创建 chroma
text_vectorstore = Chroma(
    collection_name="mm_rag_clip_photos_text",
    embedding_function=NomicEmbeddings(
        vision_model="nomic-embed-vision-v1.5", model="nomic-embed-text-v1.5"
    ),
)
image_vectorstore = Chroma(
    collection_name="mm_rag_clip_photos_image",
    embedding_function=NomicEmbeddings(
        vision_model="nomic-embed-vision-v1.5", model="nomic-embed-text-v1.5"
    ),
)

# 获取仅带有 .jpg 扩展名的图像 URI
image_uris = sorted(
    [
        os.path.join(path, image_name)
        for image_name in os.listdir(path)
        if image_name.endswith(".jpg")
    ]
)

# 添加图像
image_vectorstore.add_images(uris=image_uris)

# 添加文档
text_vectorstore.add_texts(texts=texts)

# 创建检索器
image_retriever = image_vectorstore.as_retriever()
text_retriever = text_vectorstore.as_retriever()

## RAG

`vectorstore.add_images` 将以 base64 编码字符串的形式存储/检索图像。

这些可以传递给 [GPT-4V](https://platform.openai.com/docs/guides/vision)。

In [ ]:
import base64
import io
from io import BytesIO

import numpy as np
from PIL import Image


def resize_base64_image(base64_string, size=(128, 128)):
    """
    调整以 Base64 字符串编码的图像大小。

    参数:
    base64_string (str): 原始图像的 Base64 字符串。
    size (tuple): 图像的目标大小,格式为 (宽度, 高度)。

    返回:
    str: 调整大小后的图像的 Base64 字符串。
    """
    # 解码 Base64 字符串
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # 调整图像大小
    resized_img = img.resize(size, Image.LANCZOS)

    # 将调整大小后的图像保存到字节缓冲区
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # 将调整大小后的图像编码为 Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def is_base64(s):
    """检查字符串是否为 Base64 编码"""
    try:
        return base64.b64encode(base64.b64decode(s)) == s.encode()
    except Exception:
        return False


def split_image_text_types(docs):
    """分割 numpy 数组图像和文本"""
    images = []
    text = []
    for doc in docs:
        doc = doc.page_content  # 提取文档内容
        if is_base64(doc):
            # 调整图像大小以避免 OAI 服务器错误
            images.append(
                resize_base64_image(doc, size=(250, 250))
            )  # base64 编码字符串
        else:
            text.append(doc)
    return {"images": images, "texts": text}

目前,我们使用 `RunnableLambda` 格式化输入,同时我们将图像支持添加到 `ChatPromptTemplates`。

我们的 runnable 遵循经典的 RAG 流程 - 

* 我们首先计算上下文(在这种情况下是 "texts" 和 "images")以及问题(这里只是一个 RunnablePassthrough)
* 然后我们将其传递到我们的提示模板中,这是一个自定义函数,用于格式化 gpt-4-vision-preview 模型的消息。
* 最后我们将输出解析为字符串。

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from operator import itemgetter

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


def prompt_func(data_dict):
    # 将上下文文本连接成一个字符串
    formatted_texts = "\n".join(data_dict["text_context"]["texts"])
    messages = []

    # 如果存在图像,将其添加到消息中
    if data_dict["image_context"]["images"]:
        image_message = {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{data_dict['image_context']['images'][0]}"
            },
        }
        messages.append(image_message)

    # 添加文本消息进行分析
    text_message = {
        "type": "text",
        "text": (
            "作为一名专家艺术评论家和历史学家,您的任务是分析和解释图像, "
            "考虑其历史和文化意义。除了图像外,您还将获得相关文本以提供上下文。两者都将从向量存储中检索, "
            "基于用户输入的关键字。请使用您的广泛知识和分析技能提供一个全面的总结,包括:\n"
            "- 图像中的视觉元素的详细描述。\n"
            "- 图像的历史和文化背景。\n"
            "- 图像的象征意义和含义的解释。\n"
            "- 图像与相关文本之间的联系。\n\n"
            f"用户提供的关键字: {data_dict['question']}\n\n"
            "文本和/或表格:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)

    return [HumanMessage(content=messages)]


model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)

# RAG 管道
chain = (
    {
        "text_context": text_retriever | RunnableLambda(split_image_text_types),
        "image_context": image_retriever | RunnableLambda(split_image_text_types),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(prompt_func)
    | model
    | StrOutputParser()
)

## 测试检索并运行 RAG

In [ ]:
from IPython.display import HTML, display


def plt_img_base64(img_base64):
    # 使用 base64 字符串作为源创建 HTML img 标签
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'

    # 通过渲染 HTML 显示图像
    display(HTML(image_html))


docs = text_retriever.invoke("Women with children", k=5)
for doc in docs:
    if is_base64(doc.page_content):
        plt_img_base64(doc.page_content)
    else:
        print(doc.page_content)

In [ ]:
docs = image_retriever.invoke("Women with children", k=5)
for doc in docs:
    if is_base64(doc.page_content):
        plt_img_base64(doc.page_content)
    else:
        print(doc.page_content)

In [ ]:
chain.invoke("Women with children")

我们可以在 LangSmith 跟踪中看到检索到的图像:

LangSmith [跟踪](https://smith.langchain.com/public/69c558a5-49dc-4c60-a49b-3adbb70f74c5/r/e872c2c8-528c-468f-aefd-8b5cd730a673)。